In [1]:
# Read data from csv

import gc
import pandas as pd
import catboost 
from sklearn.model_selection import train_test_split

df = pd.read_csv("~/analytics/supervised ml/Catboost_classifier/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [2]:
# Check datatypes (TotalCharges should be float64, not object)
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [3]:
# Convert TotalCharges to float64
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [4]:
# Convert object to binary value (1 & 0) for target data
df['Churn'] = df['Churn'].apply(lambda x : int(0) if x == "No" else int(1))

In [10]:
import numpy as np

# Split dataset into Feature and Target using train test split method
X = df.drop(['customerID', 'Churn'], axis=1)
y = df['Churn'] 
    
# Extract categorical feature
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [11]:
# Create functions for params auto-tuning using Optuna
# Split dataset into Feature and Target using train test split method
from sklearn.model_selection import train_test_split
from catboost import Pool
from catboost import CatBoostClassifier
import sklearn.metrics

def objective(trial):
    # Split dataset into Feature and Target using train test split method    
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3)
    train_pool = Pool(train_x, train_y, cat_features=categorical_features_indices)
    test_pool = Pool(test_x, test_y, cat_features=categorical_features_indices)

    # Fix params
    params = {
        'iterations' : trial.suggest_int('iterations', 50, 300),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait' :trial.suggest_int('od_wait', 10, 50)
    }

    # Train the model
    model = CatBoostClassifier(**params)
    model.fit(train_pool)
    
    # Do prediction
    preds = model.predict(test_pool)
    pred_labels = np.rint(preds)
    
    # Compute accuracy
    accuracy = sklearn.metrics.accuracy_score(test_y, pred_labels)
    return 1.0 - accuracy

In [12]:
# Execute Optuna
import optuna
if __name__ == '__main__':
    study = optuna.create_study()
    study.optimize(objective, n_trials=100)
    print(study.best_trial)

[I 2021-01-23 12:42:47,488] A new study created in memory with name: no-name-bbf6bef3-23b7-4cee-ad0d-12635abfc2ef


0:	learn: 0.6829047	total: 87.6ms	remaining: 6.75s
1:	learn: 0.6727599	total: 101ms	remaining: 3.85s
2:	learn: 0.6627763	total: 123ms	remaining: 3.07s
3:	learn: 0.6536042	total: 131ms	remaining: 2.42s
4:	learn: 0.6447575	total: 139ms	remaining: 2.02s
5:	learn: 0.6368428	total: 151ms	remaining: 1.81s
6:	learn: 0.6288433	total: 166ms	remaining: 1.68s
7:	learn: 0.6213272	total: 179ms	remaining: 1.57s
8:	learn: 0.6140449	total: 189ms	remaining: 1.45s
9:	learn: 0.6076471	total: 195ms	remaining: 1.33s
10:	learn: 0.6002985	total: 210ms	remaining: 1.28s
11:	learn: 0.5940104	total: 223ms	remaining: 1.23s
12:	learn: 0.5882930	total: 234ms	remaining: 1.17s
13:	learn: 0.5828448	total: 244ms	remaining: 1.11s
14:	learn: 0.5777632	total: 277ms	remaining: 1.16s
15:	learn: 0.5726281	total: 282ms	remaining: 1.09s
16:	learn: 0.5677654	total: 352ms	remaining: 1.26s
17:	learn: 0.5627559	total: 370ms	remaining: 1.23s
18:	learn: 0.5575016	total: 381ms	remaining: 1.18s
19:	learn: 0.5530057	total: 386ms	remain

[I 2021-01-23 12:42:49,200] Trial 0 finished with value: 0.207761476573592 and parameters: {'iterations': 78, 'depth': 7, 'learning_rate': 0.027797717747294615, 'random_strength': 2, 'bagging_temperature': 0.11090098327044286, 'od_type': 'Iter', 'od_wait': 11}. Best is trial 0 with value: 0.207761476573592.


69:	learn: 0.4472916	total: 1.18s	remaining: 134ms
70:	learn: 0.4462062	total: 1.2s	remaining: 118ms
71:	learn: 0.4453333	total: 1.21s	remaining: 101ms
72:	learn: 0.4443123	total: 1.23s	remaining: 84.3ms
73:	learn: 0.4438667	total: 1.24s	remaining: 67.1ms
74:	learn: 0.4430665	total: 1.27s	remaining: 50.7ms
75:	learn: 0.4421165	total: 1.28s	remaining: 33.7ms
76:	learn: 0.4413676	total: 1.29s	remaining: 16.7ms
77:	learn: 0.4407268	total: 1.29s	remaining: 0us
0:	learn: 0.6341014	total: 26.7ms	remaining: 5.83s
1:	learn: 0.6098659	total: 59.5ms	remaining: 6.45s
2:	learn: 0.5846212	total: 96.6ms	remaining: 6.96s
3:	learn: 0.5753224	total: 100ms	remaining: 5.39s
4:	learn: 0.5586404	total: 117ms	remaining: 5.01s
5:	learn: 0.5343418	total: 194ms	remaining: 6.88s
6:	learn: 0.5173415	total: 314ms	remaining: 9.51s
7:	learn: 0.5148691	total: 322ms	remaining: 8.49s
8:	learn: 0.5129372	total: 332ms	remaining: 7.74s
9:	learn: 0.5114311	total: 346ms	remaining: 7.23s
10:	learn: 0.5037350	total: 575ms	re

153:	learn: 0.2632275	total: 14.6s	remaining: 6.14s
154:	learn: 0.2624588	total: 14.7s	remaining: 6.06s
155:	learn: 0.2605087	total: 14.9s	remaining: 6s
156:	learn: 0.2577346	total: 15.1s	remaining: 5.97s
157:	learn: 0.2563203	total: 15.3s	remaining: 5.9s
158:	learn: 0.2560510	total: 15.6s	remaining: 5.89s
159:	learn: 0.2546314	total: 16s	remaining: 5.9s
160:	learn: 0.2532133	total: 16.1s	remaining: 5.81s
161:	learn: 0.2505245	total: 16.4s	remaining: 5.76s
162:	learn: 0.2485660	total: 16.5s	remaining: 5.67s
163:	learn: 0.2461595	total: 16.7s	remaining: 5.62s
164:	learn: 0.2437035	total: 16.9s	remaining: 5.53s
165:	learn: 0.2415917	total: 17.2s	remaining: 5.48s
166:	learn: 0.2407214	total: 17.4s	remaining: 5.41s
167:	learn: 0.2390480	total: 17.6s	remaining: 5.33s
168:	learn: 0.2371471	total: 17.7s	remaining: 5.23s
169:	learn: 0.2365523	total: 17.8s	remaining: 5.14s
170:	learn: 0.2343540	total: 18.2s	remaining: 5.1s
171:	learn: 0.2327877	total: 18.5s	remaining: 5.04s
172:	learn: 0.231182

...

115:	learn: 0.3302061	total: 2.1s	remaining: 90.5ms
116:	learn: 0.3291842	total: 2.13s	remaining: 72.8ms
117:	learn: 0.3283377	total: 2.15s	remaining: 54.7ms
118:	learn: 0.3282321	total: 2.18s	remaining: 36.7ms
119:	learn: 0.3275263	total: 2.21s	remaining: 18.4ms
120:	learn: 0.3267804	total: 2.24s	remaining: 0us
0:	learn: 0.6301875	total: 5.55ms	remaining: 560ms
1:	learn: 0.6038891	total: 11.3ms	remaining: 566ms
2:	learn: 0.5878058	total: 12.9ms	remaining: 427ms
3:	learn: 0.5570929	total: 20.5ms	remaining: 501ms
4:	learn: 0.5456956	total: 26.2ms	remaining: 509ms
5:	learn: 0.5287477	total: 30.1ms	remaining: 481ms
6:	learn: 0.5262347	total: 32.4ms	remaining: 440ms
7:	learn: 0.5037952	total: 36ms	remaining: 423ms
8:	learn: 0.4933567	total: 42.8ms	remaining: 442ms
9:	learn: 0.4873732	total: 47.9ms	remaining: 441ms
10:	learn: 0.4869468	total: 50.3ms	remaining: 416ms
11:	learn: 0.4866761	total: 55.8ms	remaining: 419ms
12:	learn: 0.4768512	total: 75.5ms	remaining: 517ms
13:	learn: 0.4767273	t

[I 2021-01-23 12:47:18,369] Trial 98 finished with value: 0.20350212967345005 and parameters: {'iterations': 102, 'depth': 10, 'learning_rate': 0.23027798140524544, 'random_strength': 93, 'bagging_temperature': 4.1307494926796275, 'od_type': 'Iter', 'od_wait': 45}. Best is trial 45 with value: 0.17084713677236152.


101:	learn: 0.3098736	total: 2.05s	remaining: 0us
0:	learn: 0.6341297	total: 18.6ms	remaining: 2.02s
1:	learn: 0.6079225	total: 21.3ms	remaining: 1.15s
2:	learn: 0.5813153	total: 24.8ms	remaining: 885ms
3:	learn: 0.5582054	total: 32.8ms	remaining: 870ms
4:	learn: 0.5473362	total: 37.9ms	remaining: 795ms
5:	learn: 0.5316985	total: 64.4ms	remaining: 1.12s
6:	learn: 0.5284476	total: 66.1ms	remaining: 972ms
7:	learn: 0.5168925	total: 70.4ms	remaining: 897ms
8:	learn: 0.5154835	total: 72.9ms	remaining: 818ms
9:	learn: 0.5144995	total: 79.2ms	remaining: 792ms
10:	learn: 0.5008974	total: 90.1ms	remaining: 811ms
11:	learn: 0.5004736	total: 94.7ms	remaining: 773ms
12:	learn: 0.5002045	total: 97.8ms	remaining: 730ms
13:	learn: 0.5000337	total: 99.5ms	remaining: 682ms
14:	learn: 0.4832140	total: 107ms	remaining: 679ms
15:	learn: 0.4804226	total: 115ms	remaining: 673ms
16:	learn: 0.4693492	total: 125ms	remaining: 686ms
17:	learn: 0.4599311	total: 182ms	remaining: 928ms
18:	learn: 0.4581695	total: 

[I 2021-01-23 12:47:21,013] Trial 99 finished with value: 0.21864647420728822 and parameters: {'iterations': 110, 'depth': 10, 'learning_rate': 0.2073046585646889, 'random_strength': 91, 'bagging_temperature': 3.1276600703435258, 'od_type': 'Iter', 'od_wait': 38}. Best is trial 45 with value: 0.17084713677236152.


105:	learn: 0.3172053	total: 2.3s	remaining: 86.8ms
106:	learn: 0.3160967	total: 2.33s	remaining: 65.2ms
107:	learn: 0.3153560	total: 2.36s	remaining: 43.7ms
108:	learn: 0.3140241	total: 2.39s	remaining: 21.9ms
109:	learn: 0.3133406	total: 2.42s	remaining: 0us
FrozenTrial(number=45, values=[0.17084713677236152], datetime_start=datetime.datetime(2021, 1, 23, 12, 45, 43, 753817), datetime_complete=datetime.datetime(2021, 1, 23, 12, 45, 45, 153707), params={'iterations': 128, 'depth': 5, 'learning_rate': 0.18246405249630554, 'random_strength': 30, 'bagging_temperature': 15.170616660329348, 'od_type': 'Iter', 'od_wait': 16}, distributions={'iterations': IntUniformDistribution(high=300, low=50, step=1), 'depth': IntUniformDistribution(high=10, low=4, step=1), 'learning_rate': LogUniformDistribution(high=0.3, low=0.01), 'random_strength': IntUniformDistribution(high=100, low=0, step=1), 'bagging_temperature': LogUniformDistribution(high=100.0, low=0.01), 'od_type': CategoricalDistribution(ch

In [13]:
# Show best score
study.best_value

0.17084713677236152

In [14]:
# Show Optimized params
study.best_params

{'iterations': 128,
 'depth': 5,
 'learning_rate': 0.18246405249630554,
 'random_strength': 30,
 'bagging_temperature': 15.170616660329348,
 'od_type': 'Iter',
 'od_wait': 16}